# Lab 18: RAG for Security Documentation

Build a Retrieval-Augmented Generation system for security knowledge bases.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab18_security_rag.ipynb)

## Learning Objectives
- Document chunking strategies
- Vector embeddings for semantic search
- RAG pipeline architecture
- Context-aware response generation

**Next:** Lab 21 (YARA Generator) or Lab 45 (Binary Basics primer)

In [ ]:
# Install dependencies (uncomment for Colab - use Ctrl+/ or Cmd+/ to toggle)
# !pip install chromadb sentence-transformers anthropic

# For Google Colab: Use Secret Manager for secure API key storage
# from google.colab import userdata
# import os
# os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

In [ ]:
from typing import List, Dict
import json
import hashlib

## 1. Security Knowledge Base

In [ ]:
# Comprehensive security documentation knowledge base
SECURITY_DOCS = [
    # === INCIDENT RESPONSE ===
    {
        "title": "Ransomware Response Playbook",
        "content": """When ransomware is detected, immediately isolate affected systems from the network by disabling network adapters or unplugging cables. Do not pay the ransom - this encourages attackers and doesn't guarantee decryption. Preserve evidence by capturing memory dumps using tools like WinPMem or Magnet RAM Capture, and disk images using FTK Imager. Identify patient zero and the infection vector through timeline analysis. Check backup integrity before restoration - verify backups weren't encrypted. Report to law enforcement (FBI IC3, CISA) and engage incident response specialists. Document all actions taken with timestamps. Consider threat actor TTPs to prevent reinfection.""",
        "category": "incident_response",
        "mitre_tactics": ["TA0040"],
    },
    {
        "title": "Data Breach Response Procedures",
        "content": """Upon discovering a data breach: 1) Contain the breach by revoking compromised credentials and blocking malicious IPs. 2) Assess scope - determine what data was accessed (PII, PHI, financial data, credentials). 3) Preserve evidence following chain of custody. 4) Engage legal counsel for regulatory obligations (GDPR 72-hour notification, CCPA, HIPAA). 5) Prepare notification letters for affected individuals. 6) Conduct root cause analysis to identify the vulnerability exploited. 7) Implement remediation measures. 8) Update incident response plans based on lessons learned. Consider credit monitoring services for affected individuals if financial/PII data was exposed.""",
        "category": "incident_response",
        "mitre_tactics": ["TA0010"],
    },
    {
        "title": "Malware Incident Handling",
        "content": """Malware incident handling steps: 1) Identification - confirm malware presence through AV alerts, behavioral indicators (unusual processes, network connections). 2) Containment - isolate infected systems, block C2 domains/IPs at firewall. 3) Eradication - remove malware using AV/EDR, clean registry entries, scheduled tasks, and services. 4) Recovery - restore from clean backups, reimage if necessary, reset credentials. 5) Lessons learned - document IOCs (file hashes, domains, IPs) for threat intel sharing. For fileless malware, focus on memory analysis and PowerShell logs. Always check for persistence mechanisms before declaring system clean.""",
        "category": "incident_response",
        "mitre_tactics": ["TA0002", "TA0003"],
    },
    {
        "title": "Business Email Compromise Response",
        "content": """BEC attack response: 1) Immediately contact financial institutions to attempt wire recall if funds were transferred (24-48 hour window typically). 2) Preserve email evidence including headers for forensic analysis. 3) Identify compromised accounts - check for forwarding rules, delegated access, recent password changes. 4) Reset affected account credentials and revoke all OAuth tokens. 5) Enable MFA if not already active. 6) Review mail flow rules for suspicious forwarding to external addresses. 7) Notify law enforcement and file IC3 complaint. 8) Train employees on BEC tactics including invoice manipulation, executive impersonation, and vendor fraud.""",
        "category": "incident_response",
        "mitre_tactics": ["TA0001", "TA0009"],
    },
    # === THREAT DETECTION ===
    {
        "title": "Phishing Detection Guidelines",
        "content": """Signs of phishing emails include: sender domain mismatches (paypa1.com vs paypal.com), urgency language ("Your account will be closed!"), suspicious links (hover to preview), unexpected attachments especially .zip, .docm, .js, grammar/spelling errors, and requests for credentials or financial information. Technical controls: Enable SPF (sender validation), DKIM (message integrity), and DMARC (policy enforcement) for email authentication. Deploy email gateway filtering with sandbox analysis for attachments. Implement URL rewriting and time-of-click protection. Train users with simulated phishing campaigns. Establish clear reporting procedures (phishing@company.com).""",
        "category": "threat_detection",
        "mitre_tactics": ["TA0001"],
    },
    {
        "title": "Lateral Movement Detection",
        "content": """Monitor for lateral movement indicators: PsExec usage (Event ID 7045 for PSEXESVC), WMI remote execution (Event ID 5861), RDP connections from unusual sources (Event ID 4624 Type 10), pass-the-hash attacks (4624 Type 3 without 4768), and Kerberos anomalies (4769 with RC4 encryption suggesting Kerberoasting). Key events: 4624 (Type 3 network logons), 4648 (explicit credentials used), 5140 (share access). Network indicators: SMB traffic between workstations, unusual RDP patterns, internal port scanning. Mitigations: Implement network segmentation, use jump servers/PAWs for admin access, deploy EDR with lateral movement detection, enable Windows Defender Credential Guard.""",
        "category": "threat_detection",
        "mitre_tactics": ["TA0008"],
    },
    {
        "title": "Command and Control Detection",
        "content": """C2 detection techniques: 1) Network beaconing analysis - look for regular interval connections (e.g., 60 seconds with jitter). 2) DNS anomalies - high volume of TXT queries, long subdomain names (DNS tunneling), queries to newly registered domains. 3) HTTP/S anomalies - unusual User-Agent strings, high volume POST requests, connections to IP addresses without SNI. 4) JA3/JA3S fingerprinting - identify known malware TLS fingerprints. 5) Domain analysis - algorithmically generated domains (DGA), fast-flux DNS. Deploy network detection tools (Zeek, Suricata) and integrate with threat intelligence feeds. Monitor for Cobalt Strike, Meterpreter, and commodity RAT indicators.""",
        "category": "threat_detection",
        "mitre_tactics": ["TA0011"],
    },
    {
        "title": "Credential Theft Detection",
        "content": """Credential access detection: 1) LSASS access - Sysmon Event ID 10 for process access to lsass.exe, especially with ACCESS_PROCESS_VM_READ. 2) SAM/NTDS access - Event ID 4662 for AD replication requests from non-DCs (DCSync). 3) Kerberoasting - Event ID 4769 with ticket encryption type 0x17 (RC4) for service accounts. 4) AS-REP Roasting - 4768 with 0x17 encryption for accounts with preauth disabled. 5) DPAPI access - access to CREDHIST and master key files. 6) Mimikatz indicators - process names, PE signatures, YARA rules. Deploy Credential Guard, enable LSA protection, audit Domain Admin group membership changes (4728, 4732).""",
        "category": "threat_detection",
        "mitre_tactics": ["TA0006"],
    },
    {
        "title": "Insider Threat Detection",
        "content": """Insider threat indicators: 1) Data exfiltration - large file downloads, USB usage on sensitive systems, personal cloud storage access. 2) Access anomalies - accessing files outside job scope, off-hours activity, bulk file access. 3) Behavioral changes - resignation followed by data access, bypassing controls. 4) Technical indicators - disabling logging, installing unauthorized software, using anonymizing tools. Deploy DLP solutions with content inspection. Implement UEBA for baseline deviations. Monitor privileged account usage. Establish data classification and access controls. Conduct exit interviews and access revocation procedures. Log all access to sensitive data repositories.""",
        "category": "threat_detection",
        "mitre_tactics": ["TA0010", "TA0009"],
    },
    # === ENDPOINT SECURITY ===
    {
        "title": "PowerShell Security Monitoring",
        "content": """Enable PowerShell script block logging (Event ID 4104) for full command capture and module logging (4103) for pipeline execution. Monitor for: encoded commands (-enc, -encodedcommand, -e followed by Base64), download cradles (IEX, Invoke-Expression, DownloadString, DownloadFile), AMSI bypass attempts (reflection to set amsiInitFailed), and constrained language mode bypass. Block PowerShell v2 via Feature Removal which lacks modern security features. Deploy Constrained Language Mode in production. Use AppLocker/WDAC to control PowerShell execution. Monitor for powershell.exe spawned by Office applications (WINWORD.EXE -> powershell.exe). Collect PSReadLine history (~\\AppData\\Roaming\\Microsoft\\Windows\\PowerShell\\PSReadLine).""",
        "category": "endpoint_security",
        "mitre_tactics": ["TA0002"],
    },
    {
        "title": "EDR Deployment Best Practices",
        "content": """EDR deployment guidelines: 1) Coverage - ensure all endpoints including servers, VDI, and cloud workloads are covered. 2) Policy configuration - enable all telemetry sources (process, network, file, registry), tune for environment. 3) Integration - connect to SIEM for centralized alerting, integrate with SOAR for automation. 4) Response capabilities - test isolation, remote shell, and file retrieval features. 5) Behavioral detection - enable rules for credential theft, lateral movement, persistence. 6) Exclusions - minimize to avoid coverage gaps, document all exclusions. Key features to enable: memory protection, exploit prevention, fileless attack detection. Conduct regular purple team exercises to validate detection efficacy. Monitor EDR health dashboards for offline/degraded agents.""",
        "category": "endpoint_security",
        "mitre_tactics": ["TA0005"],
    },
    {
        "title": "Windows Event Log Monitoring",
        "content": """Critical Windows events to monitor: Security: 4624/4625 (logon success/failure), 4648 (explicit creds), 4672 (admin logon), 4720/4726 (account created/deleted), 4732 (group membership), 4688 (process creation with command line), 4698 (scheduled task). System: 7045 (service installed), 7040 (service config change). PowerShell: 4104 (script block), 4103 (module). Sysmon (if deployed): 1 (process create), 3 (network), 7 (image load), 8 (CreateRemoteThread), 10 (process access), 11 (file create), 13 (registry). Enable command line logging via GPO. Configure adequate log size (1GB+ for Security). Forward to SIEM using Windows Event Forwarding (WEF) or agent-based collection.""",
        "category": "endpoint_security",
        "mitre_tactics": ["TA0005"],
    },
    {
        "title": "Endpoint Hardening Guidelines",
        "content": """Endpoint hardening checklist: 1) OS hardening - disable SMBv1, enable Attack Surface Reduction (ASR) rules, deploy Credential Guard, enable LSASS protection. 2) Application control - deploy AppLocker or WDAC, block execution from user-writable directories. 3) Patch management - prioritize critical/weaponized vulnerabilities, target 24-72 hour patch cycle for critical. 4) Local admin - remove users from local admin group, implement LAPS for unique passwords. 5) Firewall - enable Windows Firewall, block inbound RDP, limit outbound to required services. 6) USB - restrict removable storage via GPO. 7) Browser - enable SmartScreen, block risky file types, deploy browser isolation for sensitive sites. 8) Disable legacy protocols - NetBIOS, LLMNR, WPAD.""",
        "category": "endpoint_security",
        "mitre_tactics": ["TA0005"],
    },
    # === NETWORK SECURITY ===
    {
        "title": "Network Segmentation Guide",
        "content": """Network segmentation principles: 1) Identify crown jewels - databases, domain controllers, financial systems, IP repositories. 2) Segment by trust level - create zones for DMZ, corporate, servers, and restricted networks. 3) Implement zero trust between segments - assume breach, verify all access. 4) Deploy next-gen firewalls at segment boundaries with application awareness. 5) Microsegmentation - use host-based firewalls or SDN for workload isolation. Critical segments: Domain Controllers (restricted admin access only), database tier (app servers only), development (isolated from production), guest WiFi (no internal access). Log all inter-segment traffic. Regularly audit firewall rules for overly permissive access. Test segmentation with penetration testing.""",
        "category": "network_security",
        "mitre_tactics": ["TA0008"],
    },
    {
        "title": "DNS Security Configuration",
        "content": """DNS security measures: 1) DNS logging - enable query logging for all DNS servers, capture client IP, query name, response. 2) DNS sinkholing - redirect known malicious domains to internal server for detection. 3) Response Policy Zones (RPZ) - block newly registered domains, known malicious categories. 4) DNSSEC - validate responses to prevent spoofing. 5) DNS over HTTPS/TLS - consider for privacy, but ensure visibility isn't lost. 6) Threat intelligence integration - subscribe to domain blocklists, automate updates. Detection: Monitor for high-volume queries (tunneling), long subdomain names (>50 chars), queries to rare TLDs, direct queries to external resolvers bypassing corporate DNS. Deploy dedicated DNS security tools (Cisco Umbrella, Infoblox) for advanced protection.""",
        "category": "network_security",
        "mitre_tactics": ["TA0011"],
    },
    {
        "title": "Firewall Rule Management",
        "content": """Firewall best practices: 1) Default deny - explicit allow rules only, deny all at end of ruleset. 2) Least privilege - allow minimum required ports/protocols, avoid ANY rules. 3) Rule documentation - every rule needs business justification, owner, and review date. 4) Change management - all firewall changes through formal process with approval. 5) Regular review - quarterly rule audits, remove unused rules, consolidate where possible. 6) Logging - log all denied traffic, log allowed traffic for sensitive segments. 7) Testing - validate rules after changes, use automated tools for rule analysis. Red flags: Rules allowing all traffic, rules without expiration, rules to/from ANY, rules referencing deprecated systems. Implement microsegmentation for east-west traffic, not just perimeter.""",
        "category": "network_security",
        "mitre_tactics": ["TA0005"],
    },
    {
        "title": "IDS/IPS Tuning and Management",
        "content": """IDS/IPS deployment guidance: 1) Placement - deploy at network perimeter, segment boundaries, and in front of critical assets. 2) Rule management - subscribe to commercial and open source rulesets (ET Pro, Snort), tune for false positives. 3) SSL/TLS inspection - deploy inline decryption for encrypted traffic visibility, exempt sensitive categories (banking, healthcare). 4) Tuning process - run in IDS mode initially, analyze alerts, create exceptions, then enable IPS blocking. 5) Custom rules - develop rules for organization-specific threats and IOCs. 6) Integration - forward alerts to SIEM, integrate with SOAR for automated blocking. Performance: Size appropriately for throughput, monitor for dropped packets. Enable both signature and anomaly-based detection where available.""",
        "category": "network_security",
        "mitre_tactics": ["TA0005"],
    },
    # === IDENTITY & ACCESS MANAGEMENT ===
    {
        "title": "Multi-Factor Authentication Guide",
        "content": """MFA implementation priorities: 1) High-risk first - VPN, email, privileged accounts, cloud admin consoles. 2) Choose strong factors - prefer FIDO2/WebAuthn hardware keys, then authenticator apps (TOTP), avoid SMS where possible. 3) Phishing-resistant MFA - implement FIDO2 for high-value accounts to prevent real-time phishing attacks. 4) Fallback procedures - secure account recovery process, temporary bypass requires manager approval. 5) User experience - consider SSO to reduce MFA fatigue, implement risk-based authentication. 6) Monitoring - alert on MFA bypass, disabled MFA, failed MFA attempts. Common attacks to mitigate: MFA fatigue/push bombing (rate limit), real-time phishing (FIDO2), SIM swapping (no SMS MFA). Ensure MFA covers both interactive and non-interactive logons.""",
        "category": "identity_access",
        "mitre_tactics": ["TA0006"],
    },
    {
        "title": "Privileged Access Management",
        "content": """PAM implementation framework: 1) Inventory - identify all privileged accounts including service accounts, local admin, domain admin, cloud admin. 2) Vault credentials - store in PAM solution (CyberArk, Thycotic, HashiCorp Vault), rotate regularly. 3) Just-in-time access - provide temporary elevation, require approval for privileged access. 4) Session management - record and monitor privileged sessions, require jump server usage. 5) Reduce standing privilege - minimize persistent admin access, prefer temporary group membership. 6) Service accounts - use gMSA where possible, implement separate accounts per application. 7) Emergency access - define break-glass procedures with monitoring. Tiering: Tier 0 (DC, PKI) access only from PAWs, Tier 1 (servers) from secure workstations, Tier 2 (workstations) standard admin.""",
        "category": "identity_access",
        "mitre_tactics": ["TA0004"],
    },
    {
        "title": "Active Directory Security",
        "content": """AD hardening checklist: 1) Protected Users group - add sensitive accounts to prevent credential caching. 2) Credential Guard - deploy to prevent credential theft from memory. 3) LAPS - unique local admin passwords for all workstations. 4) Tiered admin model - separate accounts for workstation, server, and domain admin. 5) Kerberos - disable RC4, enable AES-only, configure S4U2self restrictions. 6) DCSync prevention - limit Replicating Directory Changes permissions. 7) GPO security - restrict who can modify GPOs, deploy LSDOU order correctly. 8) Monitoring - alert on AdminSDHolder modifications, domain admin group changes, trust creation, schema changes. Attack surface reduction: Disable print spooler on DCs, restrict NTLM, remove unnecessary domain admin accounts. Conduct regular AD security assessments using BloodHound (defensively).""",
        "category": "identity_access",
        "mitre_tactics": ["TA0006", "TA0003"],
    },
    # === CLOUD SECURITY ===
    {
        "title": "AWS Security Best Practices",
        "content": """AWS security fundamentals: 1) IAM - never use root account, enable MFA everywhere, use least privilege policies, implement permission boundaries. 2) S3 - block public access at account level, enable default encryption, enable access logging, use VPC endpoints. 3) Networking - use VPCs with private subnets, security groups (stateful), NACLs (stateless), enable VPC Flow Logs. 4) Encryption - use KMS for key management, enable encryption at rest for all services, enforce TLS. 5) Monitoring - enable CloudTrail in all regions, use GuardDuty for threat detection, set up Config rules for compliance. 6) Compute - use IMDSv2 only, deploy in private subnets, use Systems Manager for access (no bastion). Critical alerts: Root login, IAM policy changes, security group modifications, CloudTrail disabled.""",
        "category": "cloud_security",
        "mitre_tactics": ["TA0005"],
    },
    {
        "title": "Azure Security Configuration",
        "content": """Azure security essentials: 1) Identity - use Azure AD with MFA, implement PIM for just-in-time access, configure Conditional Access policies. 2) Network - deploy NSGs on all subnets, use Azure Firewall for central egress, enable DDoS protection on public resources. 3) Storage - require secure transfer (HTTPS), use private endpoints, implement SAS token best practices. 4) Monitoring - enable Azure Security Center/Defender for Cloud, configure Log Analytics workspace, enable diagnostic logs. 5) Key management - use Azure Key Vault, implement RBAC for secrets, enable soft delete. 6) Governance - use Azure Policy for compliance, deploy Blueprints for consistent configuration. Alerts to configure: Unusual sign-in locations, privileged role assignments, resource deletions in production subscriptions.""",
        "category": "cloud_security",
        "mitre_tactics": ["TA0005"],
    },
    {
        "title": "Container Security Guide",
        "content": """Container/Kubernetes security: 1) Image security - scan images for vulnerabilities in CI/CD (Trivy, Clair), use minimal base images (distroless, Alpine), sign images with Notary/Cosign. 2) Runtime - run as non-root, drop capabilities, use read-only filesystem, implement seccomp/AppArmor profiles. 3) Kubernetes - enable RBAC, use namespaces for isolation, deploy network policies, avoid hostNetwork/hostPID/hostIPC. 4) Secrets - use external secret managers (Vault, AWS Secrets Manager), never hardcode in images. 5) Monitoring - deploy Falco for runtime threat detection, enable audit logging, use admission controllers (OPA Gatekeeper). 6) Supply chain - implement SBOM, verify base images, use private registries. Critical: Disable privileged containers, limit resource consumption, implement pod security policies/standards.""",
        "category": "cloud_security",
        "mitre_tactics": ["TA0002", "TA0005"],
    },
    # === SIEM & LOG MANAGEMENT ===
    {
        "title": "SIEM Implementation Guide",
        "content": """SIEM deployment best practices: 1) Log sources - prioritize security-relevant logs: authentication (AD, VPN, SSO), network (firewall, proxy, DNS), endpoint (EDR, sysmon, AV), cloud (CloudTrail, Azure AD). 2) Normalization - map fields to common schema (OCSF, ECS) for cross-source correlation. 3) Use cases - start with high-value detections: brute force, impossible travel, lateral movement, privilege escalation. 4) Tuning - allocate resources for ongoing tuning, measure false positive rates. 5) Retention - define based on compliance (SOX 7 years, PCI 1 year), tiered storage for cost optimization. 6) Enrichment - add threat intel, geo-IP, asset context. 7) Response - integrate with SOAR for automated playbooks. KPIs: MTTD (mean time to detect), MTTR (mean time to respond), false positive rate, coverage percentage.""",
        "category": "security_operations",
        "mitre_tactics": ["TA0005"],
    },
    {
        "title": "Log Analysis Techniques",
        "content": """Log analysis methodologies: 1) Baseline establishment - understand normal patterns before detecting anomalies. 2) Timeline analysis - correlate events across sources using UTC timestamps. 3) Field extraction - parse structured data from logs using regex, grok patterns. 4) Aggregation - group by source IP, user, process to identify patterns. 5) Statistical analysis - identify outliers using standard deviation, percentiles. 6) Hunt queries - develop queries for specific TTPs (Cobalt Strike beacons, PowerShell empire). 7) IOC matching - search for known bad indicators from threat intel. Analysis workflow: Start broad, narrow based on findings, pivot on discovered entities. Key relationships to analyze: parent-child processes, network connections from process, user to system access patterns. Document findings in structured format for handoff.""",
        "category": "security_operations",
        "mitre_tactics": ["TA0005"],
    },
    # === THREAT INTELLIGENCE ===
    {
        "title": "Threat Intelligence Program",
        "content": """Threat intel program components: 1) Strategic - understand threat landscape, actor motivations, industry targeting (board reporting). 2) Tactical - TTPs and attack patterns for detection engineering (MITRE ATT&CK mapping). 3) Operational - campaigns, malware, infrastructure (hunting and response). 4) Technical - IOCs for immediate detection (hashes, IPs, domains). Sources: Commercial feeds (Mandiant, Recorded Future, Unit 42), open source (AlienVault OTX, abuse.ch), ISACs, government (CISA). Integration points: Firewall blocking, SIEM enrichment, EDR watchlists, email gateway. Lifecycle: Collection -> Processing -> Analysis -> Dissemination -> Feedback. Metrics: Intel consumption rate, detection rate from intel, time to integrate new IOCs. Quality: Validate IOCs before deployment, manage false positive feedback loop.""",
        "category": "threat_intelligence",
        "mitre_tactics": ["TA0043"],
    },
    {
        "title": "MITRE ATT&CK Framework Usage",
        "content": """MITRE ATT&CK application: 1) Detection coverage mapping - identify which techniques your controls detect, find gaps. 2) Threat profiling - map adversary groups to techniques for prioritized defense (APT29 uses T1566.001). 3) Red team planning - use Navigator layers to plan exercises covering specific techniques. 4) Detection engineering - write rules targeting technique implementations, not just tool signatures. 5) Incident analysis - categorize attacker actions using technique IDs for consistent reporting. Key matrices: Enterprise (Windows, Linux, macOS, Cloud), Mobile, ICS. Layers to maintain: Current coverage, target coverage, red team scope. Data sources: Map required visibility to technique detection (process creation for T1059). Sub-techniques: Provide specific detection opportunities (T1003.001 LSASS vs T1003.003 NTDS).""",
        "category": "threat_intelligence",
        "mitre_tactics": ["TA0043"],
    },
    # === VULNERABILITY MANAGEMENT ===
    {
        "title": "Vulnerability Management Program",
        "content": """Vulnerability management framework: 1) Asset inventory - maintain comprehensive inventory including cloud, containers, OT systems. 2) Scanning - authenticated scans weekly for critical systems, monthly for standard, continuous for cloud. 3) Prioritization - use CVSS as baseline, adjust with: exploitability (CISA KEV, EPSS), asset criticality, exposure (internet-facing). 4) Remediation SLAs - Critical: 24-72 hours, High: 7 days, Medium: 30 days, Low: 90 days. 5) Exceptions - formal risk acceptance process with compensating controls. 6) Metrics - mean time to remediate (MTTR), vulnerability density, overdue percentage. Tooling: Combine infrastructure scanning (Tenable, Qualys) with application scanning (DAST/SAST) and container scanning. Integrate with ticketing for remediation tracking. Conduct risk-based prioritization rather than attempting to fix all vulnerabilities.""",
        "category": "vulnerability_management",
        "mitre_tactics": ["TA0042"],
    },
    {
        "title": "Patch Management Process",
        "content": """Patch management best practices: 1) Inventory - maintain accurate software inventory tied to asset management. 2) Classification - categorize patches by severity, adjust for weaponized exploits. 3) Testing - test in non-production environment, automate regression testing where possible. 4) Deployment rings - pilot group (IT) -> early adopters -> broad deployment. 5) Emergency patching - process for zero-day/actively exploited (skip testing for critical). 6) Rollback - maintain ability to roll back patches, test restore procedures. 7) Metrics - patch compliance rate, patch latency (time from release to deployment). Automation: Use WSUS/SCCM/Intune for Windows, Ansible/Chef/Puppet for Linux, cloud-native patching for cloud instances. Third-party applications often more risky than OS - prioritize browser, PDF readers, Java. Monthly patch cycle aligned with Patch Tuesday typical for most organizations.""",
        "category": "vulnerability_management",
        "mitre_tactics": ["TA0042"],
    },
]

print(f"📚 Loaded {len(SECURITY_DOCS)} security documents")
print("\n📊 Documents by Category:")
category_counts = {}
for doc in SECURITY_DOCS:
    cat = doc["category"]
    category_counts[cat] = category_counts.get(cat, 0) + 1
for cat, count in sorted(category_counts.items(), key=lambda x: -x[1]):
    print(f"   {cat.replace('_', ' ').title()}: {count}")

print(f"\n📋 Sample Topics Covered:")
for doc in SECURITY_DOCS[:5]:
    print(f"   • {doc['title']}")

## 2. Simple Vector Store

In [ ]:
import numpy as np


class SimpleVectorStore:
    """Simple in-memory vector store using TF-IDF-like scoring."""

    def __init__(self):
        self.documents = []
        self.vocabulary = set()

    def add_documents(self, docs: List[Dict]):
        """Add documents to the store."""
        for doc in docs:
            text = doc["content"].lower()
            words = set(text.split())
            self.vocabulary.update(words)
            self.documents.append(
                {
                    "title": doc["title"],
                    "content": doc["content"],
                    "words": words,
                    "category": doc.get("category", "general"),
                }
            )

    def search(self, query: str, top_k: int = 3) -> List[Dict]:
        """Search for relevant documents."""
        query_words = set(query.lower().split())

        scores = []
        for doc in self.documents:
            # Simple word overlap scoring
            overlap = len(query_words & doc["words"])
            score = overlap / (len(query_words) + 0.1)
            scores.append((score, doc))

        # Sort by score
        scores.sort(key=lambda x: x[0], reverse=True)

        return [{"score": s, "document": d} for s, d in scores[:top_k]]


# Initialize store
store = SimpleVectorStore()
store.add_documents(SECURITY_DOCS)
print(f"Indexed {len(store.documents)} documents")

## 3. RAG Pipeline

In [ ]:
class SecurityRAG:
    """RAG system for security documentation."""

    def __init__(self, vector_store):
        self.store = vector_store
        try:
            from anthropic import Anthropic

            self.client = Anthropic()
            self.available = True
        except:
            self.available = False

    def query(self, question: str) -> str:
        """Answer a question using RAG."""
        # Retrieve relevant documents
        results = self.store.search(question, top_k=3)

        # Build context
        context = "\n\n".join(
            [
                f"Document: {r['document']['title']}\n{r['document']['content']}"
                for r in results
                if r["score"] > 0
            ]
        )

        if not self.available:
            return self._mock_response(question, results)

        prompt = f"""Use the following security documentation to answer the question.
If the answer isn't in the documentation, say so.

DOCUMENTATION:
{context}

QUESTION: {question}

Provide a clear, actionable answer."""

        response = self.client.messages.create(
            model="claude-sonnet-4.5",
            max_tokens=512,
            messages=[{"role": "user", "content": prompt}],
        )

        return response.content[0].text

    def _mock_response(self, question: str, results: List) -> str:
        if not results or results[0]["score"] == 0:
            return "I couldn't find relevant information in the security documentation."

        doc = results[0]["document"]
        return f"""Based on the {doc['title']}:

{doc['content']}

---
(Retrieved from: {doc['category']} documentation)"""

## Understanding RAG: How It Works

### The RAG Pipeline Explained

```
┌─────────────────────────────────────────────────────────────────┐
│                      RAG PIPELINE                                │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   User Question: "How do I respond to ransomware?"              │
│         │                                                        │
│         ▼                                                        │
│   ┌─────────────────────┐                                       │
│   │  RETRIEVAL PHASE    │                                       │
│   │  Search documents   │                                       │
│   │  for relevant info  │                                       │
│   └──────────┬──────────┘                                       │
│              │                                                   │
│              ▼                                                   │
│   Retrieved: [Ransomware Playbook, Malware Handling, ...]       │
│              │                                                   │
│              ▼                                                   │
│   ┌─────────────────────┐                                       │
│   │  AUGMENTATION       │                                       │
│   │  Inject documents   │                                       │
│   │  into LLM prompt    │                                       │
│   └──────────┬──────────┘                                       │
│              │                                                   │
│              ▼                                                   │
│   Prompt: "Using this context: [docs] Answer: [question]"       │
│              │                                                   │
│              ▼                                                   │
│   ┌─────────────────────┐                                       │
│   │  GENERATION         │                                       │
│   │  LLM creates answer │                                       │
│   │  grounded in docs   │                                       │
│   └──────────┬──────────┘                                       │
│              │                                                   │
│              ▼                                                   │
│   Answer: "When ransomware is detected, immediately..."         │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### Why RAG Matters for Security

1. **Grounded Answers**: LLM responses are based on YOUR documentation, not generic training data
2. **Reduced Hallucination**: Model can only cite what's in the retrieved docs
3. **Up-to-Date**: Add new playbooks without retraining the model
4. **Citation**: Can point to exact source documents

### Evaluating RAG Quality

When reviewing RAG output, check:

| Aspect | Good Sign | Red Flag |
|--------|-----------|----------|
| **Relevance** | Answer addresses the question | Generic or off-topic response |
| **Grounding** | Information matches source docs | Claims not in retrieved docs |
| **Completeness** | Covers key points from docs | Missing important procedures |
| **Actionability** | Clear next steps provided | Vague or theoretical advice |

### Common RAG Failure Modes

1. **Retrieval Failure**: Wrong documents retrieved
   - *Solution*: Better embeddings, more documents, query expansion
   
2. **Context Overflow**: Too much context confuses LLM
   - *Solution*: Better chunking, smarter document selection
   
3. **Hallucination Despite Context**: LLM ignores retrieved docs
   - *Solution*: Prompt engineering, lower temperature

In [ ]:
# Test RAG system with diverse security questions
rag = SecurityRAG(store)

questions = [
    # Incident Response
    "What should I do when ransomware is detected?",
    "How do I respond to a business email compromise attack?",
    # Threat Detection
    "How can I detect lateral movement in my network?",
    "What are the signs of credential theft?",
    # Endpoint Security
    "How do I monitor PowerShell for attacks?",
    "What Windows events should I monitor for security?",
    # Network Security
    "How should I implement network segmentation?",
    "How do I configure DNS security?",
    # Identity & Access
    "What is the best approach for multi-factor authentication?",
    "How do I secure Active Directory?",
    # Cloud Security
    "What are AWS security best practices?",
    "How do I secure containers and Kubernetes?",
]

print("=" * 60)
print("Security RAG System - Query Demo")
print("=" * 60)

for q in questions[:6]:  # Show first 6 for brevity
    print(f"\n🔍 Q: {q}")
    print("-" * 50)
    answer = rag.query(q)
    # Truncate long answers for display
    if len(answer) > 500:
        print(answer[:500] + "...")
    else:
        print(answer)
    print()

print(f"\n📊 RAG System Statistics:")
print(f"   Total Documents: {len(SECURITY_DOCS)}")
print(f"   Categories: {len(category_counts)}")
print(f"   Questions Tested: {len(questions)}")

## Summary

We built a RAG system for security documentation with:

1. **Document Store** - Indexed security playbooks and guides
2. **Search** - Simple word-overlap scoring (production: use embeddings)
3. **Generation** - Context-aware answers from LLM

### Next Steps:
1. Use sentence-transformers for semantic embeddings
2. Add ChromaDB or Pinecone for vector storage
3. Implement document chunking for large docs